In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../raw_data/df_energy_weather.csv", index_col=[0])
#print(df.columns)

#pd.options.display.max_columns
pd.set_option("display.max_columns", None)
df.columns

Index(['Date', 'Code INSEE région', 'Consommation (MW)', 'Thermique (MW)',
       'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)',
       'Pompage (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre',
       'Eolien offshore', 'TCO Thermique (%)', 'TCH Thermique (%)',
       'TCO Nucléaire (%)', 'TCH Nucléaire (%)', 'TCO Eolien (%)',
       'TCH Eolien (%)', 'TCO Solaire (%)', 'TCH Solaire (%)', 'Column 30',
       'YEAR', 'MONTH', 'DAY', 'T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE',
       'T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M',
       'WS50M', 'sin_month', 'cos_month', 'week_day', 'sin_day', 'cos_day'],
      dtype='object')

In [3]:
df = df[['Consommation (MW)', 'T2M', 'TS', 'week_day','sin_month', 'cos_month','sin_day', 'cos_day']]

In [4]:
len_ = int(0.9*df.shape[0])
df_train = df[:len_]
#df_train = df[:1014]
df_test = df[len_:]
df_train

,Consommation (MW),T2M,TS,week_day,sin_month,cos_month,sin_day,cos_day
0,252371.0,7.30,7.08,1,5.000000e-01,0.866025,0.781831,0.623490
1,280064.0,4.36,4.10,2,5.000000e-01,0.866025,0.974928,-0.222521
2,285526.0,5.46,3.78,3,5.000000e-01,0.866025,0.433884,-0.900969
3,286799.0,5.92,4.06,4,5.000000e-01,0.866025,-0.433884,-0.900969
4,267848.0,6.64,4.85,5,5.000000e-01,0.866025,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...
3070,181843.0,19.89,20.30,5,5.000000e-01,-0.866025,-0.974928,-0.222521
3071,174427.0,19.35,20.33,6,5.000000e-01,-0.866025,-0.781831,0.623490
3072,184765.0,19.48,20.56,0,5.000000e-01,-0.866025,0.000000,1.000000
3073,187841.0,18.69,19.26,1,1.224647e-16,-1.000000,0.781831,0.623490


In [5]:
from sklearn.preprocessing import MinMaxScaler
columns = df.columns

scaler = MinMaxScaler()
df_train = pd.DataFrame(scaler.fit_transform(df_train), columns=columns)
df_test = pd.DataFrame(scaler.transform(df_test), columns=columns)


In [6]:
from tensorflow.keras import models
from tensorflow.keras import layers, metrics
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
def train_model(input_shape_value):
    metric = metrics.MAPE

    

    model = models.Sequential()
    #model.add(layers.Masking(mask_value=-1000., input_shape=(30,2)))
    #model.add(layers.LSTM(20, return_sequences=True, activation='tanh'))
    # Ajout d'une couche + drop


    model.add(layers.LSTM(20, return_sequences = True, input_shape=input_shape_value, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(30, return_sequences = True, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(20, return_sequences = True, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(10, return_sequences = False, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(5, activation = 'relu'))
    model.add(layers.Dense(1, activation = 'linear'))


    model.compile(loss='mse', 
                  optimizer=RMSprop(learning_rate=0.0003), 
                  metrics=[metric])
    
    

    return model


In [8]:
def grid(df_train,df_test,nb_jours_entrainement):
    steps = 1
    #nb_jours_entrainement = 360
    nb_jours_prevision = 14
    df_train_shape = df_train.shape[0]
    df_test_shape = df_test.shape[0]
    
    X_train = []
    y_train = []

    for i in range(nb_jours_entrainement, df_train_shape - nb_jours_prevision, steps):
        X_train.append(df_train[i-nb_jours_entrainement:i])
        #y_test.append(df_train[i : i + nb_jours_prevision,0:1])
        #.iloc[:, 1:2]
        y_train.append(df_train.iloc[i : i + nb_jours_prevision, 0:1])
        
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    input_shape_value = X_train.shape[1:3]
    
    model = train_model(input_shape_value)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
 
    model.fit(X_train, y_train,
            validation_split=0.2,
            epochs=1000, 
            batch_size=32,
            callbacks=[es], verbose=1)
    
    X_test = []
    y_test = []

    for i in range(nb_jours_entrainement, df_test_shape - nb_jours_prevision, steps):
        X_test.append(df_test[i-nb_jours_entrainement:i])
        #y_test.append(df_train[i : i + nb_jours_prevision,0:1])
        #.iloc[:, 1:2]
        y_test.append(df_test.iloc[i : i + nb_jours_prevision, 0:1])
    
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    res = model.evaluate(X_test, y_test, verbose=1)
    
    return res

In [9]:
list_nb_jours_entrainement = [300,420,510]

In [10]:
for nb_jours_entrainement in list_nb_jours_entrainement:
    res = grid(df_train,df_test,nb_jours_entrainement)
    print(f'taille_sequence = {nb_jours_entrainement}// LOSS = {res[0]} // MAPE = {res[1]}')

2022-06-17 17:59:29.747436: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-17 17:59:30.276009: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
69/69 [==============================] - 13s 149ms/step - loss: 0.0277 - mean_absolute_percentage_error: 42.5646 - val_loss: 0.0104 - val_mean_absolute_percentage_error: 245547.9531
Epoch 2/1000
69/69 [==============================] - 10s 141ms/step - loss: 0.0161 - mean_absolute_percentage_error: 35.1951 - val_loss: 0.0110 - val_mean_absolute_percentage_error: 292862.8438
Epoch 3/1000
69/69 [==============================] - 10s 144ms/step - loss: 0.0138 - mean_absolute_percentage_error: 31.6706 - val_loss: 0.0088 - val_mean_absolute_percentage_error: 238106.9219
Epoch 4/1000
69/69 [==============================] - 10s 141ms/step - loss: 0.0119 - mean_absolute_percentage_error: 29.3515 - val_loss: 0.0079 - val_mean_absolute_percentage_error: 235281.4375
Epoch 5/1000
69/69 [==============================] - 10s 140ms/step - loss: 0.0107 - mean_absolute_percentage_error: 27.3189 - val_loss: 0.0096 - val_mean_absolute_percentage_error: 292942.7812
Epoch 6/1000
69/69 [=====

/Users/thomasbordier/.pyenv/versions/3.8.13/envs/forecast_energy_consumption/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:909: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer